In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd


In [4]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df=df.dropna()

In [9]:
X=df.drop('label',axis=1)
y=df['label']


In [10]:
from keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM
from keras.layers import Dense

In [11]:
voc_size=5000

In [12]:
messages=X.copy()

In [13]:
messages.reset_index(inplace=True)


In [14]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
type(corpus)

list

In [17]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2455, 887, 3947, 1328, 1351, 231, 3032, 1470, 4696, 4358],
 [2079, 4409, 2345, 746, 2262, 1115, 3896],
 [3839, 3896, 2279, 4743],
 [3936, 2735, 2151, 3865, 4294, 1540],
 [164, 2262, 1558, 1410, 1551, 799, 2262, 1847, 4307, 2676],
 [306,
  1686,
  4816,
  4068,
  1711,
  2115,
  2066,
  581,
  1836,
  337,
  4344,
  383,
  558,
  2719,
  3896],
 [331, 35, 4821, 4455, 3266, 2088, 1582, 900, 2545, 3184, 2288],
 [3860, 4897, 4398, 1977, 3528, 2096, 2115, 4329, 2545, 3184, 2288],
 [2861, 1639, 4603, 4978, 868, 176, 331, 2085, 2115, 57],
 [2127, 245, 4223, 2312, 1830, 2657, 1108, 4654],
 [3041, 1989, 4756, 2336, 2897, 1597, 4073, 754, 1943, 773, 693],
 [3865, 4969, 1351, 176, 2115, 3528],
 [3070, 4476, 294, 2038, 3747, 95, 392, 732, 2002],
 [4879, 3319, 3639, 636, 1203, 3274, 303, 2545, 3184, 2288],
 [861, 4271, 1895, 4682, 3168, 2545, 3184, 2288],
 [1804, 3188, 3535, 2640, 4799, 1644, 4677, 1137, 4637, 3613],
 [430, 845, 4409],
 [2838, 3754, 1182, 550, 2115, 2083, 3514, 3896],
 [3828, 288

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2455  887 3947 ...    0    0    0]
 [2079 4409 2345 ...    0    0    0]
 [3839 3896 2279 ...    0    0    0]
 ...
 [ 840 4201  969 ...    0    0    0]
 [2341 3528 1207 ...    0    0    0]
 [1703  490 3771 ...    0    0    0]]


In [24]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 24s 111ms/step - loss: 0.3164 - accuracy: 0.8450 - val_loss: 0.1985 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1459 - accuracy: 0.9433 - val_loss: 0.1891 - val_accuracy: 0.9241
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1017 - accuracy: 0.9633 - val_loss: 0.1990 - val_accuracy: 0.9176
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0760 - accuracy: 0.9724 - val_loss: 0.2417 - val_accuracy: 0.9176
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0569 - accuracy: 0.9793 - val_loss: 0.2679 - val_accuracy: 0.9085
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0464 - accuracy: 0.9840 - val_loss: 0.2853 - val_accuracy: 0.9120
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0357 - accuracy: 0.9876 - val_loss: 0.3088 - val_accuracy: 0.914

In [28]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve


189/189 [==============================] - 1s 3ms/step


In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(y_test,y_pred)

array([[3206,  213],
       [ 439, 2177]])

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.891963545981773

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      3419
           1       0.91      0.83      0.87      2616

    accuracy                           0.89      6035
   macro avg       0.90      0.88      0.89      6035
weighted avg       0.89      0.89      0.89      6035

